In [1]:
import napari
import numpy as np
import matplotlib.pyplot as plt
import z5py
from pathlib import Path

from cryofib.n5_utils import read_volume, print_key_tree, write_volume
from cryofib.napari_utils import napari_add_group

## View n5 datasets

In [2]:
data_dir = Path("/g/kreshuk/buglakova/data/cryofib/segm_fibsem/F107")
em_n5_path = data_dir / "F107_A1_em.n5"

f_em = z5py.File(em_n5_path, "r")

In [3]:
print_key_tree(f_em)

Key structure of z5 file /g/kreshuk/buglakova/data/cryofib/segm_fibsem/F107/F107_A1_em.n5
raw
segmentation
segmentation/nuclei
segmentation/edoardo
segmentation/edoardo/boundaries
segmentation/edoardo/foreground
segmentation/edoardo/junctions
segmentation/ground_truth_channels
segmentation/ground_truth
segmentation/corrected_instances
raw_norm


## Read volumes

In [4]:
roi = np.s_[:]
em = read_volume(f_em, "raw", roi)

<class 'z5py.file.File'>
Reading roi slice(None, None, None) of volume raw from /g/kreshuk/buglakova/data/cryofib/segm_fibsem/F107/F107_A1_em.n5
Read volume with shape (1350, 1506, 1647), data type uint8


In [5]:
em_segm = read_volume(f_em, "segmentation/corrected_instances", roi)

<class 'z5py.file.File'>
Reading roi slice(None, None, None) of volume segmentation/corrected_instances from /g/kreshuk/buglakova/data/cryofib/segm_fibsem/F107/F107_A1_em.n5
Read volume with shape (1350, 1506, 1647), data type int64


In [6]:
junctions_segm = read_volume(f_em, "segmentation/edoardo/junctions", roi)

<class 'z5py.file.File'>
Reading roi slice(None, None, None) of volume segmentation/edoardo/junctions from /g/kreshuk/buglakova/data/cryofib/segm_fibsem/F107/F107_A1_em.n5
Read volume with shape (1350, 1506, 1647), data type uint8


## Open raw in napari

In [6]:
v = napari.Viewer()

In [7]:
ds = f_em["raw"]
resolution = ds.attrs["resolution"]
v.add_image(em, name="raw")

<Image layer 'raw' at 0x7f68104d77c0>

In [9]:
v.add_labels(junctions_segm, name="junctions")

<Labels layer 'junctions' at 0x7f38a4146590>

In [8]:
v.add_labels(em_segm, name="em_segm")

<Labels layer 'em_segm' at 0x7f68103411b0>

In [7]:
dict(ds.attrs)

{'description': 'Raw FIB-SEM data F107_A1',
 'resolution': [40, 30, 30],
 'roi': {'start': [0, 0, 0], 'step': [1, 1, 1], 'stop': [10, 1506, 1647]},
 'unit': 'nm'}

## Save corrected annotations

In [58]:
cur_annotation = v.layers["em_segm"].data
write_volume(em_n5_path, cur_annotation, "segmentation/corrected_instances")

<class 'pathlib.PosixPath'>
Overwriting segmentation/corrected_instances
Writing array to segmentation/corrected_instances


In [26]:
np.max(cur_annotation)

29582

### Open all in a group

In [8]:
napari_add_group(f_em, "segmentation", v)